In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#from IPython.display import Image
from scipy import stats
import numpy as np

## Initial value problem---Example

$$\frac{du}{dt} = -a(\omega)u$$
$$u(t=0,\omega) = b$$
$$a \sim N(a_0,\sigma_a^2)$$
The damping rate a is random. 

In [ ]:
a0 = 1
sigma_a = 0.25
b0 = 10

The analytical solution is $$u(t) = b e^{-at}$$

In [ ]:
def u(b,a,t): #Exact solution
    return b*np.exp(-a*t)

$$E[u(t)] = b e^{-a_0 t} e^{\sigma_a^2 t^2/2}$$
$$var[u(t)] = e^{-2a_0 t} b^2 (e^{2 \sigma_a^2 t^2} - e^{\sigma_a^2 t^2})$$

In [ ]:
t = np.linspace(0, 12, 100)
umean_exact = b0*np.exp(-a0*t)*np.exp(sigma_a**2*t**2/2)
uvar_exact = b0**2*np.exp(-2*a0*t)*(np.exp(2*sigma_a**2*t**2)-np.exp(sigma_a**2*t**2))

## Direct simulation

In [ ]:
nt=100
t2 = t.reshape(nt,1)
fig, ax  = plt.subplots()
ax.plot(t,u(b0,np.random.normal(a0,sigma_a,100),t2));

In [ ]:
udirect = u(b0,np.random.normal(a0,sigma_a,10),t2)
umean = np.mean(udirect,axis=1)
uplus = umean + 2*np.std(udirect,axis=1)
uminus = umean - 2*np.std(udirect,axis=1)
fig, ax  = plt.subplots()
ax.plot(t,umean)
ax.plot(t,uplus)
ax.plot(t,uminus)
ax.plot(t,umean_exact,'k--')
ax.plot(t,umean_exact+2*np.sqrt(uvar_exact),'k--')
ax.plot(t,umean_exact-2*np.sqrt(uvar_exact),'k--')
plt.xlabel('Time (s)'),plt.ylabel('Displacement (m)')
plt.legend(['Mean'])
plt.title('2-$\sigma$ credible intervals')

## Stochastic Spectral

In [ ]:
K=8

In [ ]:
def e_ink(i,n,k):
    s2 = i + n + k
    s = (i + n + k)/2
    if np.mod(s2,2)==1:
        f = 0
    elif ((s<i) | (s<n) | (s<k)):
        f = 0
    else:
        f = np.math.factorial(i)*np.math.factorial(n)*np.math.factorial(k)/np.math.factorial(s-i)/np.math.factorial(s-n)/np.math.factorial(s-k)
    return f

In [ ]:
A = np.zeros(shape=(K+1,K+1))
gamma = np.zeros(K+1)
for i in range(K+1):
    gamma[i] = np.math.factorial(i)
    for k in range(K+1):
        A[i,k] = -1/np.math.factorial(i)*(a0*e_ink(i,0,k)+sigma_a*e_ink(i,1,k))

In [ ]:
plt.pcolor(A)

In [ ]:
from scipy.integrate import odeint
def dU_dt(U, t, A):
    # Here U is a vector such that y=U[0] and z=U[1]. This function should return [y', z']
    return A.dot(U)

In [ ]:
U0 = np.zeros(K+1)
U0[0] = b0
UK = odeint(dU_dt, U0, t, args=(A,))
UKmean = UK[:,0]
UKvar = np.sum(gamma[1:]*UK[:,1:]**2,axis=1)

In [ ]:
fig, ax  = plt.subplots()
ax.plot(t,UKmean)
ax.plot(t,UKmean + 2*np.sqrt(UKvar))
ax.plot(t,UKmean - 2*np.sqrt(UKvar))
ax.plot(t,umean_exact,'k--')
ax.plot(t,umean_exact+2*np.sqrt(uvar_exact),'k--')
ax.plot(t,umean_exact-2*np.sqrt(uvar_exact),'k--')

## Direct projection

In [ ]:
from numpy.polynomial import HermiteE as H

In [ ]:
qq = np.linspace(-2, 2, 100)
for i in range(4): ax = plt.plot(qq, H.basis(i)(qq), lw=2, label="$H_%d$"%i)
plt.legend(loc="lower left")

$$u_k(t) = \frac{1}{\gamma_k} \sum_{r=1}^R u(t,q^r) \Psi_k(q^r) \rho_Q(q^r) w^r$$
Use Gauss-Hermite quadrature points

In [ ]:
R = 12
q,w = np.polynomial.hermite_e.hermegauss(R)
w = w/np.sqrt(2*np.pi)
np.sum(w)

In [ ]:
k=0
plt.plot(np.sum(H.basis(k)(q)*w*u(b0,a0+sigma_a*q,t2),axis=1)/gamma[k])

In [ ]:
UKp = np.zeros(shape=(nt,K+1))
for k in range(K+1):
    UKp[:,k] = np.sum(H.basis(k)(q)*w*u(b0,a0+sigma_a*q,t2),axis=1)/gamma[k]

In [ ]:
UKpmean = UKp[:,0]
UKpvar = np.sum(gamma[1:]*UKp[:,1:]**2,axis=1)

In [ ]:
fig, ax  = plt.subplots()
ax.plot(t,UKpmean)
ax.plot(t,UKpmean + 2*np.sqrt(UKpvar))
ax.plot(t,UKpmean - 2*np.sqrt(UKpvar))
ax.plot(t,umean_exact,'k--')
ax.plot(t,umean_exact+2*np.sqrt(uvar_exact),'k--')
ax.plot(t,umean_exact-2*np.sqrt(uvar_exact),'k--')